In [4]:
import numpy as np
import matplotlib.pyplot as plt

class IsingMonteCarlo:
    def __init__(self, L=20, dilution=0.0):
        self.L = L
        self.N = L * L
        self.dilution = dilution
        
        # Initialize spins: Random +1 or -1
        self.spins = np.random.choice([-1, 1], size=(L, L))
        
        # Create a defect mask (1 = spin exists, 0 = empty site)
        if dilution > 0:
            mask_indices = np.random.rand(L, L) > dilution
            self.mask = mask_indices.astype(int)
            self.spins *= self.mask # Apply dilution
        else:
            self.mask = np.ones((L, L), dtype=int)

    def energy(self):
        """Calculate total energy of the lattice (Standard Ising Hamiltonian)"""
        # Periodic boundary conditions handled by np.roll
        neighbors = (
            np.roll(self.spins, 1, axis=0) + 
            np.roll(self.spins, -1, axis=0) + 
            np.roll(self.spins, 1, axis=1) + 
            np.roll(self.spins, -1, axis=1)
        )
        # Multiply by mask to ensure empty sites don't contribute
        E = -np.sum(self.spins * neighbors) / 2
        return E

    def mc_step(self, T):
        """Metropolis-Hastings Step"""
        for _ in range(self.N):
            # Pick random site
            i, j = np.random.randint(0, self.L, 2)
            
            # If site is a defect (empty), skip
            if self.mask[i, j] == 0:
                continue
                
            s = self.spins[i, j]
            
            # Sum of neighbors
            nb = (self.spins[(i+1)%self.L, j] + 
                  self.spins[(i-1)%self.L, j] + 
                  self.spins[i, (j+1)%self.L] + 
                  self.spins[i, (j-1)%self.L])
            
            dE = 2 * s * nb
            
            # Metropolis criterion
            if dE < 0 or np.random.rand() < np.exp(-dE / T):
                self.spins[i, j] *= -1
                
    def simulate(self, T, steps=1000, equilibrium=500):
        """Run simulation for a specific Temperature"""
        # Equilibration
        for _ in range(equilibrium):
            self.mc_step(T)
            
        configs = []
        # Data Collection
        for _ in range(steps):
            self.mc_step(T)
            # Save copy of current config
            configs.append(self.spins.copy())
            
        return np.array(configs)

# --- Example of Generating Data ---
def generate_dataset(L=20, dilution=0.0, temps=None):
    if temps is None:
        temps = np.linspace(1.0, 4.0, 30)
    
    X_data = []
    Y_temp = [] # Storing Temperature for plotting later
    
    print(f"Generating data for L={L}, Dilution={dilution}...")
    
    for T in temps:
        # Re-initialize system for each T to avoid getting stuck in meta-stable states
        sim = IsingMonteCarlo(L=L, dilution=dilution)
        
        # Run MC
        # We take fewer snapshots per T for this demo, increase 'steps' for real training
        configs = sim.simulate(T, steps=50, equilibrium=500) 
        
        X_data.append(configs)
        Y_temp.extend([T] * len(configs))
        
    X_data = np.vstack(X_data)
    Y_temp = np.array(Y_temp)
    
    return X_data, Y_temp

# 1. Generate Training Data (PURE SYSTEM)
# We need labeled data: Low T (Label 0) and High T (Label 1)
# 2D Ising Tc is approx 2.269
train_low_T = generate_dataset(L=20, dilution=0.0, temps=[1.0, 1.5]) 
train_high_T = generate_dataset(L=20, dilution=0.0, temps=[3.5, 4.0])

X_train = np.vstack([train_low_T[0], train_high_T[0]])
# Labels: 0 for Ordered, 1 for Disordered
Y_train = np.concatenate([np.zeros(len(train_low_T[0])), np.ones(len(train_high_T[0]))])

# Shuffle
p = np.random.permutation(len(X_train))
X_train = X_train[p]
Y_train = Y_train[p]

print(f"Training Data Shape: {X_train.shape}, Labels Shape: {Y_train.shape}")

Generating data for L=20, Dilution=0.0...
Generating data for L=20, Dilution=0.0...
Training Data Shape: (200, 20, 20), Labels Shape: (200,)


In [3]:
Y_train

array([1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1.,
       1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
       1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0.,
       0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1.,
       1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
       0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
       1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0.,
       1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
       0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.])